In [39]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import interpolate

In [2]:
from bender_extras import make_motor_stepper_pulses

In [3]:
%load_ext autoreload
%autoreload 2

# Generate signal

Sampling parameters

In [42]:
samplingfreq = 1000.0
outputfreq = 100000.0

waitbefore = 4.0
waitafter = 1.0

## Movement parameters

In [43]:
freq = 2.2     # Hz
amp = 10       # deg
ncycles = 10

## Activation parameters

In [44]:
activation_duty = 0.3       # fractions of a cycle
activation_phase = 0.1      # fractions of a cycle

activation_pulse_rate = 30  # Hz
start_cycle = 3

is_activation = True        # set to False for passive tests

In [45]:
actburstdur = activation_duty / freq

# make sure the activation is an even number of pulses
actburstdur = np.floor(actburstdur * activation_pulse_rate * 2) / (
        activation_pulse_rate * 2)

actburstduty = actburstdur * freq

print("Activation burst duration: {:.3} sec".format(actburstdur))
print("True activation duty: {:.3}".format(actburstduty))

Activation burst duration: 0.133 sec
True activation duty: 0.293


Start setting up the output

In [46]:
movedur = ncycles / freq
totaldur = waitbefore + movedur + waitafter

t = np.arange(0, totaldur, 1.0/samplingfreq)
t -= waitbefore

tnorm = t * freq

Generate the angle and angular velocity signals

In [47]:
angle = amp * np.sin(2*np.pi * freq * t)
anglevel = 2*np.pi * amp * freq * np.cos(2*np.pi * freq * t)

angle[t < 0] = 0
angle[t > movedur] = 0

anglevel[t < 0] = 0
anglevel[t > movedur] = 0

In [54]:
actcmd = np.zeros_like(t)
Lonoff = []
Ronoff = []

if is_activation:
    pulsedur = 0.01         # 10ms long pulse to start the S88

    startstim = np.zeros((int(actburstdur * samplingfreq + 1),))
    startstim[:int(pulsedur * samplingfreq)] = 5

    bendphase = tnorm - 0.25

    # list of cycles when we'll have activation
    actcycles = list(range(start_cycle, ncycles))

    # also do one cycle before the bending starts
    actcycles.insert(0, -3)
    # and one after
    actcycles.append(ncycles+1)

    for c in actcycles:
        k = np.argmax(bendphase >= c + activation_phase)
        tstart = t[k]
        tend = tstart + actburstdur

        if any(bendphase >= c + activation_phase):
            Lonoff.append([tstart, tend])
        if any(bendphase >= c + activation_phase + 0.5):
            Ronoff.append(np.array([tstart, tend]) + 0.5 / freq)

        np.place(actcmd, np.logical_and(bendphase >= c + activation_phase,
                                        bendphase < c + activation_phase + actburstduty),
                                        startstim)

Lonoff = np.array(Lonoff)
Ronoff = np.array(Ronoff)
actcmdhi = interpolate.interp1d(t, actcmd, kind='linear', assume_sorted=True, bounds_error=False,
                                    fill_value=0.0)(tout)

and plot them:

In [59]:
for onoff in Lonoff:
    print(onoff)

[-1.204      -1.07066667]
[1.523      1.65633333]
[1.978      2.11133333]
[2.432      2.56533333]
[2.887      3.02033333]
[3.341      3.47433333]
[3.796      3.92933333]
[4.25       4.38333333]
[5.16       5.29333333]


In [70]:
fig = make_subplots(rows = 2, cols = 1,
                   shared_xaxes=True)
fig.add_trace(
    go.Scatter(x = t, y = angle, mode="lines", name="angle"),
    row=1, col=1)
fig.add_trace(
    go.Scatter(x = t, y = actcmd, mode="lines", name="act"),
    row=1, col=1)

for onoff in Lonoff:
    fig.add_vrect(x0 = onoff[0], x1=onoff[1], fillcolor="black", opacity=0.25, line_width=0,
                      row=1, col=1)

for onoff in Ronoff:
    fig.add_vrect(x0 = onoff[0], x1=onoff[1], opacity=0.7, line_width=1,
                      row=1, col=1)

fig.update_yaxes(title_text = "angle (deg)", row=1)
fig.add_trace(
    go.Scatter(x = t, y = anglevel, mode="lines", name="anglevel"),
    row=2, col=1)

fig.update_yaxes(title_text = "angular velocity (deg/s)", row=2)
fig.update_xaxes(title_text = "time (s)", row=2)

Generate the motor step and direction pulses. 

In [9]:
tout, dig, step, direction = make_motor_stepper_pulses(t, angle, anglevel, outputfreq)

Use the cell below to debug the step and direction pulses, but don't render it every time, because it takes a long time to plot the traces, because the output sampling rate is high.

In [13]:
# fig = make_subplots()
# fig.add_trace(go.Scatter(x=tout, y=step, mode="lines", name="step"))
# fig.add_trace(go.Scatter(x=tout, y=direction, mode="lines", name="dir"))